# filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 집계 조건을 만족하는 Group의 행 조회
    1. DataFrameGroupBy의 group로 DataFrame을 함수에 전달
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교 후 반환(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환
- 매개변수
    - func: filtering 조건을 구현한 함수
        - 첫번째 매개변수로 Group으로 묶인 DataFrame
    - dropna=True
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop, False로 설정 시 NA 반환
    - \*args, \*\*kwargs: filter 함수의 매개변수에 전달할 전달 인자값

In [1]:
import numpy as np
import pandas as pd

In [2]:
# cnt1 - 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


#### 과일중 cnt1의 평균이 20 이상인 과일들만 보기
- 집계결과가 아니라 조건을 만족하는 행들을 다 출력

In [3]:
# 과일 별 평균 조회
r = df.groupby('fruits').mean()
# 평균이 20 이상인 과일 결과 조회
r[r['cnt1'] >=  20]  # 귤, 딸기
# 귤과 딸기의 정보 조회
df[(df['fruits'] == '귤') | (df['fruits'] == '딸기')] 

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [4]:
# groupby.filter(함수)
def ck_cnt1_mean(X):
    '''
    filter에 사용할 함수
    매개변수 : X (group 별로 분류된 DataFrame)
    반환값 : bool (특정조건 만족 여부)
    '''
    return X['cnt1'].mean() >= 20

# group 별 특정 조건을 만족하는 행들을 조회하는 함수
df.groupby('fruits').filter(ck_cnt1_mean)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [5]:
# lambda식 구현
df.groupby('fruits').filter(lambda X : X['cnt1'].mean() >= 20)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [6]:
# dropna= : 결과를 만족하지 못한 항목 출력 여부, True : 미출력(기본값), False : 출력
df.groupby('fruits').filter(lambda X : X['cnt1'].mean() >= 20, dropna=False)

,fruits,cnt1,cnt2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,귤,21.0,51.0
6,귤,22.0,57.0
7,귤,27.0,58.0
8,귤,24.0,57.0
9,귤,26.0,51.0


#### 매개변수 있는 filter함수

In [7]:
def ck_mean(X, col_name, mean_thresh):
    '''
    [parameter]
        - X : DataFrame (그룹별 분류)
        - col_name : str(평균 계산할 컬럼명)
        - mean_thresh : int (평균 값과 비교할 비교값)
    [return]
        - bool : 컬럼의 평균이 mean_thresh 이상인 경우
    '''
    return X[col_name].mean() >= mean_thresh

In [8]:
df.groupby('fruits').filter(ck_mean, col_name='cnt2', mean_thresh=100)

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


# transform
함수에 의해 처리된 값(반환값)으로 원래 값들을 변경(tranform) 해서 반환    
DataFrame에 Group 단위 통계량을 추가 시 때 유용
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy.transform(func, *args)`
    - func: 매개변수로 그룹별로 Series를 받아 Series의 값들을 변환하여 (Series로)반환하는 함수객체
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달 (위치기반 argument)
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성 가능
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용 가능

In [9]:
# agg(), filter(), transform() - DataFrameGroupby의 메소드 : pandas제공 함수는 문자열, 사용자 정의 함수는 함수 객체 사용
df.groupby('fruits').transform('mean')

,cnt1,cnt2
0,11.6,103.6
1,11.6,103.6
2,11.6,103.6
3,11.6,103.6
4,11.6,103.6
5,24.0,54.8
6,24.0,54.8
7,24.0,54.8
8,24.0,54.8
9,24.0,54.8


In [10]:
# 사용자 정의 함수를 이용한 transform
def max_min_diff(X):
    '''
    사용자 정의 함수
    매개변수 : Series (그룹 별 분류)
    반환 : 처리한 값
    '''
    
    return X.max() - X.min()

In [11]:
df.groupby('fruits').transform(max_min_diff)

,cnt1,cnt2
0,3,7
1,3,7
2,3,7
3,3,7
4,3,7
5,6,7
6,6,7
7,6,7
8,6,7
9,6,7


In [12]:
# lambda식을 이용한 transform
df.groupby('fruits').transform(lambda X: X.max() - X.min())

,cnt1,cnt2
0,3,7
1,3,7
2,3,7
3,3,7
4,3,7
5,6,7
6,6,7
7,6,7
8,6,7
9,6,7


## 원본에 통계치 붙여서 비교하기

In [13]:
# 그룹 별 통계치를 DataFrame 행별로 붙이기
cnt1_mean = df.groupby('fruits')['cnt1'].transform('mean')

In [14]:
# 비교 대상 옆 렬에 붙이기
df.insert(2, 'cnt1 mean', cnt1_mean) # .insert(삽입할 컬럼 위치, 컬럼명, 대입할 컬럼 값) = 원본 변경
df

,fruits,cnt1,cnt1 mean,cnt2
0,사과,10,11.6,100
1,사과,12,11.6,103
2,사과,13,11.6,107
3,사과,11,11.6,107
4,사과,12,11.6,101
5,귤,21,24.0,51
6,귤,22,24.0,57
7,귤,27,24.0,58
8,귤,24,24.0,57
9,귤,26,24.0,51


In [15]:
# sample() : 표본 추출 , frac= (샘플비율 0 ~ 1) 
df = df.sample(frac=1).reset_index(drop=True) # .reset_index(drop=True) : 섞인 index 명을 다시 자동번호 명명
df

,fruits,cnt1,cnt1 mean,cnt2
0,딸기,41,34.2,206
1,배,8,5.4,5
2,딸기,30,34.2,208
3,사과,12,11.6,101
4,귤,21,24.0,51
5,귤,24,24.0,57
6,사과,13,11.6,107
7,딸기,35,34.2,217
8,딸기,37,34.2,213
9,귤,26,24.0,51


In [16]:
df['cnt2_mean'] = df.groupby('fruits')['cnt2'].transform('mean')
df

,fruits,cnt1,cnt1 mean,cnt2,cnt2_mean
0,딸기,41,34.2,206,209.6
1,배,8,5.4,5,7.4
2,딸기,30,34.2,208,209.6
3,사과,12,11.6,101,103.6
4,귤,21,24.0,51,54.8
5,귤,24,24.0,57,54.8
6,사과,13,11.6,107,103.6
7,딸기,35,34.2,217,209.6
8,딸기,37,34.2,213,209.6
9,귤,26,24.0,51,54.8


## 결측치 처리
- 결측치 : 모르는 값, 없는 값
- 결측치 처리
    1. 제거(행/열 단위)
        - 데이터 양이 충분할 경우 제거
    1. 대체 
        - 저장 가능성이 높은 값을 대체(평균, 중앙값, 최빈값)
        - 결측치를 표현하는 값으로 대체 (없다라는 값으로 변경)
- transform이용해서 여기선 결측치를 같은 과일그룹의 평균값으로 변환
    - 전체 평균보다 좀 더 정확할 수 있음

In [17]:
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)

df.loc[[0, 1, 5, 7, 10, 12, 15, 16], 'cnt2'] = None

df

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,57.0
7,귤,27,NaN
8,귤,24,57.0
9,귤,26,51.0


In [18]:
# 결측치 제거
df.dropna() # 결측치가 있는 행 제거

,fruits,cnt1,cnt2
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
6,귤,22,57.0
8,귤,24,57.0
9,귤,26,51.0
11,배,7,9.0
13,배,3,7.0
14,배,2,7.0
17,딸기,37,213.0


In [19]:
# 결측치 제거
df.dropna(axis=1) # 결측치가 있는 열 제거

,fruits,cnt1
0,사과,10
1,사과,12
2,사과,13
3,사과,11
4,사과,12
5,귤,21
6,귤,22
7,귤,27
8,귤,24
9,귤,26


In [20]:
# 대체 : fillna(대체값)
df2 = df.copy()
df2['cnt2'] = df['cnt2'].fillna(round(df['cnt2'].mean(), 1))
df2

,fruits,cnt1,cnt2
0,사과,10,93.8
1,사과,12,93.8
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,93.8
6,귤,22,57.0
7,귤,27,93.8
8,귤,24,57.0
9,귤,26,51.0


In [21]:
# 동일한 size의 Series를 넣을 경우 동일한 index 번호의 값을 대체
df['cnt2'].fillna(df.groupby('fruits')['cnt2'].transform('mean'), inplace=True)
df

,fruits,cnt1,cnt2
0,사과,10,105.000000
1,사과,12,105.000000
2,사과,13,107.000000
3,사과,11,107.000000
4,사과,12,101.000000
5,귤,21,55.000000
6,귤,22,57.000000
7,귤,27,55.000000
8,귤,24,57.000000
9,귤,26,51.000000


# TODO 
- data/diamonds.csv 조회

In [22]:
d = pd.read_csv('data/diamonds.csv')

In [23]:
# cut 별 pirce의 평균이 4000 이상인 diamond 데이터들 조회
d.groupby('cut').filter(lambda X : X['price'].mean() >= 4000)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
...,...,...,...,...,...,...,...,...,...,...
53928,0.79,Premium,E,SI2,61.4,58.0,2756,6.03,5.96,3.68
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53934,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58


In [24]:
# color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 모든 diamond 데이터들 조회
d.groupby('color').filter(lambda X : (X['carat'].max()-X['carat'].min() >= 2) & (X['carat'].max()-X['carat'].min() < 3))

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
...,...,...,...,...,...,...,...,...,...,...
53929,0.71,Ideal,G,VS1,61.4,56.0,2756,5.76,5.73,3.53
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53932,0.70,Very Good,E,VS2,60.5,59.0,2757,5.71,5.76,3.47


In [25]:
# clarity 별 평균 가격 컬럼을 DataFrame에 추가.
r = d.groupby('clarity')['price'].transform('mean')
d.insert(7, 'clarity price mean', r)

In [26]:
d

,carat,cut,color,clarity,depth,table,price,clarity price mean,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,5063.028606,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3996.001148,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,3839.455391,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,3924.989395,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,5063.028606,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,3996.001148,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,3996.001148,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,3996.001148,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,5063.028606,6.15,6.12,3.74


# pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼을 행과 열로 위치시키고 집계값을 값으로 출력    
groupby() 를 이용한 집계와 동일한 역할

> pivot() 함수와 역할이 다름   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수

- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`
- **매개변수**
    - index
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - columns
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 됨)
    - values
        - 문자열 또는 리스트. 집계할 대상 컬럼들
    - aggfunc
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략 시): 평균 (mean이 기본값)
    - fill_value, dropna
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - margins/margins_name
        - margin: boolean(기본: False). 총집계결과를 만들지 여부
        - margin_name: margin의 이름 문자열로 지정 (생략시 All)

In [27]:
fls = pd.read_csv('data/flights.csv')

In [28]:
# 2개 이상의 컬럼을 group으로 묶어 집계
# AIRLINE, MONTH 별 AIR_TIME 평균
fls.groupby(['AIRLINE', 'MONTH'])['AIR_TIME'].mean()

AIRLINE  MONTH
AA       1        138.793103
         2        143.126560
         3        142.026163
         4        144.544267
         5        145.261538
                     ...    
WN       7        109.377500
         8        109.173248
         9        109.842239
         11       104.704244
         12       104.136304
Name: AIR_TIME, Length: 149, dtype: float64

## 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [29]:
#groupby
fls.groupby('AIRLINE')['AIR_TIME'].mean()

AIRLINE
AA    144.259404
AS    147.845052
B6    209.412963
DL    115.334187
EV     68.964016
F9    127.592337
HA    338.288288
MQ     61.318346
NK    135.736878
OO     76.010272
UA    155.650521
US    147.686755
VX    154.864097
WN    107.005897
Name: AIR_TIME, dtype: float64

In [30]:
# pivot_table()
fls.pivot_table(values='AIR_TIME', # 집계대상
                index='AIRLINE',   # index 대상 컬럼
                aggfunc='mean',    # 집계함수
                margins=True       # 총 집계
               )

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [31]:
# 다수의 집계값/집계대상컬럼/index/columns 넣을 경우 리스크로 묶어 전달
fls.pivot_table(values=['AIR_TIME', 'ARR_DELAY', 'DEP_DELAY'], index='AIRLINE', aggfunc=['mean', 'median', 'max'], margins=True)

mean                         median                      \
           AIR_TIME  ARR_DELAY  DEP_DELAY AIR_TIME ARR_DELAY DEP_DELAY   
AIRLINE                                                                  
AA       144.259404   5.542661  11.274057    132.0      -5.0      -2.0   
AS       147.845052  -0.833333   1.808594    131.0      -6.0      -5.0   
B6       209.412963   8.692593  14.287823    256.0      -2.0       0.0   
DL       115.334187   0.339691   7.242450     93.0      -7.0      -1.0   
EV        68.964016   7.034580   9.092625     65.0      -3.0      -2.0   
F9       127.592337  13.630651  14.310398    118.0      -1.0      -1.0   
HA       338.288288   4.972973   2.571429    336.0       0.0      -4.0   
MQ        61.318346   6.860591  11.094323     53.0      -4.0      -1.0   
NK       135.736878  18.436070  19.514401    127.0       2.0       0.0   
OO        76.010272   7.593463   9.662279     67.0      -3.0      -2.0   
UA       155.650521   7.765755  16.210485    136.0      -4.0       1.0   
US       147.686755   1.681105   4.361355    132.0      -4.0      -3.0   
VX       154.864097   5.348884   8.918946    105.5      -3.0      -1.0   
WN       107.005897   6.397353  12.639366     93.0      -3.0       1.0   
All      115.928576   5.812315  10.857501     97.0      -4.0      -1.0   

             max                      
        AIR_TIME ARR_DELAY DEP_DELAY  
AIRLINE                               
AA         513.0     858.0     835.0  
AS         402.0     344.0     338.0  
B6         344.0     331.0     348.0  
DL         577.0     741.0     755.0  
EV         212.0     669.0     672.0  
F9         296.0     839.0     852.0  
HA         411.0     298.0     298.0  
MQ         199.0     357.0     342.0  
NK         295.0     474.0     461.0  
OO         258.0     724.0     735.0  
UA         542.0    1185.0    1194.0  
US         413.0     431.0     439.0  
VX         347.0     236.0     253.0  
WN         321.0     493.0     476.0  
All        577.0    1185.0    1194.0

## 두개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 조회)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [32]:
# groupby
fls.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [33]:
# pivot_table()
fls.pivot_table(values='CANCELLED',
                index='AIRLINE', columns='ORG_AIR', # 그룹으로 묶을 컬럼들의 index/columns 지정
                aggfunc='sum',
                margins=True
                # fill_value=0 #NaN을 지정 값으로 변경 
               )
# NaN : 그 그룹들의 값을 가진 행이 없는 경우

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,All
AIRLINE,,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0,154
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
B6,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0,38
EV,18.0,6.0,27.0,36.0,NaN,NaN,6.0,53.0,0.0,NaN,146
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,10
HA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0
MQ,5.0,NaN,62.0,0.0,NaN,0.0,0.0,85.0,NaN,NaN,152
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,NaN,25


In [34]:
# 다수의 집계값/집계대상컬럼/index/columns 넣을 경우 리스크로 묶어 전달
fls.pivot_table(values=['DIVERTED', 'CANCELLED'], index='AIRLINE', columns='ORG_AIR', aggfunc='sum')

CANCELLED                                                    DIVERTED  \
ORG_AIR       ATL   DEN   DFW   IAH  LAS   LAX  MSP   ORD  PHX   SFO      ATL   
AIRLINE                                                                         
AA            3.0   4.0  86.0   3.0  3.0  11.0  3.0  35.0  4.0   2.0      1.0   
AS            0.0   0.0   0.0   0.0  0.0   0.0  0.0   0.0  0.0   0.0      0.0   
B6            NaN   0.0   0.0   NaN  0.0   0.0  NaN   0.0  0.0   1.0      NaN   
DL           28.0   1.0   0.0   0.0  1.0   1.0  4.0   0.0  1.0   2.0     18.0   
EV           18.0   6.0  27.0  36.0  NaN   NaN  6.0  53.0  0.0   NaN      6.0   
F9            0.0   2.0   1.0   0.0  1.0   1.0  1.0   4.0  0.0   0.0      0.0   
HA            NaN   NaN   NaN   NaN  0.0   0.0  NaN   NaN  0.0   0.0      NaN   
MQ            5.0   NaN  62.0   0.0  NaN   0.0  0.0  85.0  NaN   NaN      0.0   
NK            1.0   1.0   6.0   0.0  1.0   1.0  3.0  10.0  2.0   NaN      0.0   
OO            3.0  25.0   2.0  10.0  0.0  15.0  4.0  41.0  9.0  33.0      0.0   
UA            2.0   9.0   1.0  23.0  3.0   6.0  2.0  25.0  3.0  19.0      0.0   
US            0.0   0.0   2.0   2.0  1.0   0.0  0.0   6.0  7.0   3.0      0.0   
VX            NaN   NaN   NaN   NaN  0.0   3.0  NaN   0.0  NaN   3.0      NaN   
WN            9.0  13.0   NaN   NaN  7.0  32.0  1.0   NaN  6.0  25.0      1.0   

                                                      
ORG_AIR  DEN  DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO  
AIRLINE                                               
AA       1.0  8.0  0.0  3.0  1.0  0.0  4.0  4.0  4.0  
AS       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
B6       0.0  0.0  NaN  1.0  0.0  NaN  1.0  0.0  0.0  
DL       1.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0  0.0  
EV       0.0  3.0  4.0  NaN  NaN  1.0  1.0  0.0  NaN  
F9       0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  
HA       NaN  NaN  NaN  0.0  1.0  NaN  NaN  0.0  0.0  
MQ       NaN  2.0  0.0  NaN  0.0  0.0  3.0  NaN  NaN  
NK       0.0  2.0  1.0  1.0  0.0  1.0  0.0  0.0  NaN  
OO       5.0  0.0  2.0  0.0  5.0  3.0  3.0  1.0  2.0  
UA       1.0  0.0  6.0  0.0  1.0  0.0  6.0  0.0  5.0  
US       0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  
VX       NaN  NaN  NaN  0.0  0.0  NaN  0.0  NaN  1.0  
WN       2.0  NaN  NaN  7.0  2.0  0.0  NaN  3.0  0.0

## 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [35]:
# groupby
fls.groupby(['AIRLINE', 'MONTH', 'ORG_AIR'])['CANCELLED'].sum()

AIRLINE  MONTH  ORG_AIR
AA       1      ATL        0
                DEN        0
                DFW        8
                IAH        0
                LAS        0
                          ..
WN       12     LAS        1
                LAX        2
                MSP        0
                PHX        0
                SFO        0
Name: CANCELLED, Length: 1133, dtype: int64

In [36]:
# pivot_table()
fls.pivot_table(values='CANCELLED', index=['AIRLINE', 'ORG_AIR'], columns='MONTH', aggfunc='sum', margins=True)

MONTH                1      2     3     4     5      6     7     8     9  \
AIRLINE ORG_AIR                                                            
AA      ATL        0.0    2.0   1.0   0.0   0.0    0.0   0.0   0.0   0.0   
        DEN        0.0    1.0   0.0   0.0   2.0    0.0   0.0   0.0   0.0   
        DFW        8.0   33.0  13.0   4.0   8.0    7.0   1.0   2.0   1.0   
        IAH        0.0    1.0   0.0   0.0   1.0    0.0   0.0   1.0   0.0   
        LAS        0.0    2.0   0.0   0.0   0.0    0.0   1.0   0.0   0.0   
...                ...    ...   ...   ...   ...    ...   ...   ...   ...   
WN      LAX        3.0    2.0   3.0   2.0   1.0    0.0   9.0   4.0   3.0   
        MSP        1.0    0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   
        PHX        0.0    2.0   1.0   1.0   0.0    0.0   1.0   0.0   0.0   
        SFO        4.0    5.0   0.0   2.0   2.0    6.0   0.0   2.0   2.0   
All              104.0  195.0  94.0  58.0  81.0  100.0  56.0  44.0  17.0   

MONTH              11    12  All  
AIRLINE ORG_AIR                   
AA      ATL       0.0   0.0    3  
        DEN       0.0   1.0    4  
        DFW       3.0   6.0   86  
        IAH       0.0   0.0    3  
        LAS       0.0   0.0    3  
...               ...   ...  ...  
WN      LAX       3.0   2.0   32  
        MSP       0.0   0.0    1  
        PHX       1.0   0.0    6  
        SFO       2.0   0.0   25  
All              51.0  81.0  881  

[115 rows x 12 columns]

## 3개 이상의 컬럼을 grouping해서 집계 2
- 항공사/월/출발공항코드 별 최대/최소 연착시간
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - ARR_DELAY: 연착시간
- 집계 : min, max    

In [37]:
# groupby
fls.groupby(['AIRLINE', 'MONTH', 'ORG_AIR'])['ARR_DELAY'].agg('min', 'max')

AIRLINE  MONTH  ORG_AIR
AA       1      ATL       -27.0
                DEN       -13.0
                DFW       -39.0
                IAH       -23.0
                LAS       -32.0
                           ... 
WN       12     LAS       -52.0
                LAX       -30.0
                MSP       -23.0
                PHX       -30.0
                SFO       -20.0
Name: ARR_DELAY, Length: 1133, dtype: float64

In [38]:
# pivot_table()
fls.pivot_table(values='ARR_DELAY', index=['AIRLINE', 'ORG_AIR'], columns='MONTH', aggfunc=['min', 'max'], margins=True)

min                                                        \
MONTH               1     2     3     4     5     6     7     8     9    11   
AIRLINE ORG_AIR                                                               
AA      ATL     -27.0 -26.0 -32.0 -30.0 -26.0 -23.0 -27.0 -32.0 -33.0 -33.0   
        DEN     -13.0 -17.0 -19.0 -28.0 -20.0 -13.0 -30.0 -27.0 -27.0 -27.0   
        DFW     -39.0 -29.0 -29.0 -37.0 -36.0 -33.0 -32.0 -32.0 -33.0 -45.0   
        IAH     -23.0 -27.0 -13.0 -27.0 -19.0 -13.0 -19.0 -30.0 -31.0 -14.0   
        LAS     -32.0 -25.0 -24.0 -19.0 -31.0 -27.0 -40.0 -28.0 -32.0 -31.0   
...               ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
WN      LAX     -25.0 -27.0 -25.0 -37.0 -26.0 -30.0 -25.0 -20.0 -30.0 -28.0   
        MSP     -38.0 -32.0 -30.0 -33.0 -24.0 -24.0 -28.0 -21.0 -23.0 -29.0   
        PHX     -38.0 -45.0 -24.0 -43.0 -27.0 -25.0 -29.0 -24.0 -25.0 -41.0   
        SFO     -19.0 -24.0 -29.0 -29.0 -22.0 -26.0 -30.0 -22.0 -33.0 -23.0   
All             -57.0 -58.0 -47.0 -53.0 -51.0 -50.0 -46.0 -51.0 -48.0 -57.0   

                 ...    max                                                   \
MONTH            ...      3      4      5      6      7      8      9     11   
AIRLINE ORG_AIR  ...                                                           
AA      ATL      ...   25.0  115.0   25.0  159.0  319.0   84.0  196.0  255.0   
        DEN      ...   53.0  105.0  330.0   10.0   67.0  257.0  152.0  146.0   
        DFW      ...  234.0  275.0  285.0  602.0  203.0  268.0  241.0  349.0   
        IAH      ...   97.0  127.0  131.0  456.0  858.0   95.0   73.0   98.0   
        LAS      ...  111.0  626.0   54.0  206.0  157.0  157.0   36.0   89.0   
...              ...    ...    ...    ...    ...    ...    ...    ...    ...   
WN      LAX      ...  104.0  165.0  244.0  244.0  248.0  221.0  157.0   87.0   
        MSP      ...   43.0   64.0   13.0   88.0   84.0   45.0   23.0  114.0   
        PHX      ...   78.0  130.0  203.0  168.0  171.0  161.0  131.0  244.0   
        SFO      ...  237.0   72.0  133.0  205.0   51.0   65.0   69.0  143.0   
All              ...  589.0  669.0  692.0  616.0  858.0  741.0  623.0  682.0   

                                 
MONTH                12     All  
AIRLINE ORG_AIR                  
AA      ATL       203.0   319.0  
        DEN       106.0   330.0  
        DFW       293.0   602.0  
        IAH       103.0   858.0  
        LAS       219.0   732.0  
...                 ...     ...  
WN      LAX       493.0   493.0  
        MSP        90.0   128.0  
        PHX       254.0   254.0  
        SFO       215.0   237.0  
All              1185.0  1185.0  

[115 rows x 24 columns]

# apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리 가능

- DataFrame.apply(함수, axis=0, args=())
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달
    - 매개변수
        - 함수: DataFrame의 행들 또는 열들을 전달할 함수
        - axis: **0-행을 전달, 1-열을 전달 (기본값 0)** G: 0이 행
        - args: 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
- Series.apply(함수, args=())
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달
    - 매개변수
        - 함수: Series의 원소들을 전달할 함수
        - args: 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달

In [39]:
arr = np.arange(24).reshape(6,4)
df = pd.DataFrame(arr, columns=['NO.1', 'NO.2', 'NO.3', 'NO.4'])
df

,NO.1,NO.2,NO.3,NO.4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [40]:
# apply에 전달할 함수
def func(X):
    '''
    매개변수 X
    - dataframe.apply() : Series가 전달 (DF의 한 행 또는 한 열)
    - series.apply() : Series 원소가 전달
    '''
    print(X) # 컬럼 별 조회
    print('-' * 20)
    return X ** 2

In [41]:
# apply()에 적용
df.apply(func)

0     0
1     4
2     8
3    12
4    16
5    20
Name: NO.1, dtype: int32
--------------------
0     1
1     5
2     9
3    13
4    17
5    21
Name: NO.2, dtype: int32
--------------------
0     2
1     6
2    10
3    14
4    18
5    22
Name: NO.3, dtype: int32
--------------------
0     3
1     7
2    11
3    15
4    19
5    23
Name: NO.4, dtype: int32
--------------------


,NO.1,NO.2,NO.3,NO.4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [42]:
# 행단위 적용
df.apply(func, axis=1)

NO.1    0
NO.2    1
NO.3    2
NO.4    3
Name: 0, dtype: int32
--------------------
NO.1    4
NO.2    5
NO.3    6
NO.4    7
Name: 1, dtype: int32
--------------------
NO.1     8
NO.2     9
NO.3    10
NO.4    11
Name: 2, dtype: int32
--------------------
NO.1    12
NO.2    13
NO.3    14
NO.4    15
Name: 3, dtype: int32
--------------------
NO.1    16
NO.2    17
NO.3    18
NO.4    19
Name: 4, dtype: int32
--------------------
NO.1    20
NO.2    21
NO.3    22
NO.4    23
Name: 5, dtype: int32
--------------------


,NO.1,NO.2,NO.3,NO.4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [43]:
# 매개변수가 있는 경우
def func2(X, value):
    return X + value

In [44]:
df.apply(func2, value=100)

,NO.1,NO.2,NO.3,NO.4
0,100,101,102,103
1,104,105,106,107
2,108,109,110,111
3,112,113,114,115
4,116,117,118,119
5,120,121,122,123


In [45]:
# lambda식 표현
df.apply(lambda X: X * 2)

,NO.1,NO.2,NO.3,NO.4
0,0,2,4,6
1,8,10,12,14
2,16,18,20,22
3,24,26,28,30
4,32,34,36,38
5,40,42,44,46


In [46]:
# Series에 적용
df['NO.1'].apply(func)

0
--------------------
4
--------------------
8
--------------------
12
--------------------
16
--------------------
20
--------------------


0      0
1     16
2     64
3    144
4    256
5    400
Name: NO.1, dtype: int64

In [47]:
# lambda식 표현
df['NO.1'].apply(lambda X: X + 2)

0     2
1     6
2    10
3    14
4    18
5    22
Name: NO.1, dtype: int64

# cut()/qcut() - 연속형(실수)을 범주형으로 변환

## cut()
- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶음
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - x: 범주형으로 바꿀 대상. 1차원 배열형태(Series, 리스트, ndarray)의 자료구조
        - bins: 범주로 나눌때의 기준값(구간경계)들을 리스트로 묶어서 전달
        - right: 구간경계의 오른쪽(True-기본) 포함, 왼쪽(False) 포함 여부
        - labels: 각 구간(범주)의 label을 리스트로 전달
            - 생략하면 범위를 범주명으로 사용 (ex: (10, 20], ()-포함안함, []-포함)

In [48]:
# 예제
age = pd.Series(np.random.randint(50, size=30)) # 0 ~ 50 중 임의 수 30개 반환
age

0     10
1     38
2     42
3      4
4     14
5     43
6     42
7      7
8     23
9      2
10    35
11     6
12    28
13    27
14    13
15    30
16    38
17    47
18    31
19    21
20     8
21    27
22    43
23    19
24    39
25    36
26    45
27    19
28    34
29     9
dtype: int32

In [49]:
# age의 각 index의 값이 어느 그룹에 속한지 반환
# () : 미포함, [포함]
# (-1, 10] : -1 미포함, 10 포함 (-1 < x <= 10)
bins = [-1, 10, 20, 30, 40, 50] 
labels = ['10세 이하', '10대', '20대', '30대', '40대']
age_cate = pd.cut(age, bins=bins, labels = labels, right=False)
age_cate

0        10대
1        30대
2        40대
3     10세 이하
4        10대
5        40대
6        40대
7     10세 이하
8        20대
9     10세 이하
10       30대
11    10세 이하
12       20대
13       20대
14       10대
15       30대
16       30대
17       40대
18       30대
19       20대
20    10세 이하
21       20대
22       40대
23       10대
24       30대
25       30대
26       40대
27       10대
28       30대
29    10세 이하
dtype: category
Categories (5, object): ['10세 이하' < '10대' < '20대' < '30대' < '40대']

In [50]:
# DateFrame 화
ages = pd.DataFrame({'age':age, 'age_cate':age_cate})
ages

,age,age_cate
0,10,10대
1,38,30대
2,42,40대
3,4,10세 이하
4,14,10대
5,43,40대
6,42,40대
7,7,10세 이하
8,23,20대
9,2,10세 이하


In [51]:
# 통계치 조회
ages.groupby('age_cate')['age'].mean()

age_cate
10세 이하     6.000000
10대       15.000000
20대       25.200000
30대       35.125000
40대       43.666667
Name: age, dtype: float64

## qcut
- qcut() :  대상배열의 최대값 ~ 최소값을 지정한 개수의 동등한 size(**원소의개수**)가 되도록 나눔
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - x: 나눌 대상. 1차원 배열형태의 자료구조
        - q: 나눌 개수
        - labels: 각 구간(범주)의 label을 리스트로 전달

In [52]:
# n 등분으로 지정 시 동일한 원소의 개수 n개의 그룹으로 지정
pd.qcut(age, 3)

0      (1.999, 19.0]
1     (35.333, 47.0]
2     (35.333, 47.0]
3      (1.999, 19.0]
4      (1.999, 19.0]
5     (35.333, 47.0]
6     (35.333, 47.0]
7      (1.999, 19.0]
8     (19.0, 35.333]
9      (1.999, 19.0]
10    (19.0, 35.333]
11     (1.999, 19.0]
12    (19.0, 35.333]
13    (19.0, 35.333]
14     (1.999, 19.0]
15    (19.0, 35.333]
16    (35.333, 47.0]
17    (35.333, 47.0]
18    (19.0, 35.333]
19    (19.0, 35.333]
20     (1.999, 19.0]
21    (19.0, 35.333]
22    (35.333, 47.0]
23     (1.999, 19.0]
24    (35.333, 47.0]
25    (35.333, 47.0]
26    (35.333, 47.0]
27     (1.999, 19.0]
28    (19.0, 35.333]
29     (1.999, 19.0]
dtype: category
Categories (3, interval[float64, right]): [(1.999, 19.0] < (19.0, 35.333] < (35.333, 47.0]]

In [53]:
age_cate2 = pd.qcut(age, 3, labels=['상', '중', '하'])

In [54]:
ages2 = pd.DataFrame({'age':age, 'age_cate2':age_cate2})
ages2

,age,age_cate2
0,10,상
1,38,하
2,42,하
3,4,상
4,14,상
5,43,하
6,42,하
7,7,상
8,23,중
9,2,상


In [55]:
ages2.groupby('age_cate2')['age'].count()

age_cate2
상    11
중     9
하    10
Name: age, dtype: int64

# TODO

In [56]:
# TODO 1: data/diamonds.csv 를 읽어 DataFrame으로 생성
d = pd.read_csv('data/diamonds.csv')

In [57]:
# TODO 2: price 컬럼을 '고가', '중가', '저가' 세개의 범주값을 가지는 "price_cate" 컬럼을 생성한다
pc = pd.qcut(d['price'], 3, labels=['저가', '중가', '고가'])
d.insert(7, 'price_cate', pc)
d

,carat,cut,color,clarity,depth,table,price,price_cate,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,저가,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,저가,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,저가,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,저가,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,저가,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,중가,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,중가,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,중가,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,중가,6.15,6.12,3.74


In [58]:
# TODO 3 가격대(price_cate) 별 carat의 평균을 조회
d.groupby('price_cate')['carat'].mean()

price_cate
저가    0.352691
중가    0.709712
고가    1.331734
Name: carat, dtype: float64

In [59]:
# TODO 4 가격대(price_cate)와 cut별 평균 가격(price)를 피봇테이블로 조회
d.pivot_table(values='price', index='cut', columns='price_cate', aggfunc='mean', margins=True)

price_cate,저가,중가,고가,All
cut,,,,
Fair,934.774038,2758.364720,8008.701599,4358.757764
Good,722.535903,2750.656315,7876.938825,3928.864452
Ideal,808.090128,2393.391611,8636.215870,3457.541970
Premium,814.953663,2638.198339,8669.363940,4584.257704
Very Good,710.980713,2608.592897,8231.963420,3981.759891
All,784.692932,2552.586840,8462.692603,3932.799722


In [60]:
# TODO 5 cut, color, price_cate 별 carat의 평균을 피봇테이블로 조회
d.pivot_table(values='carat', index= ['cut', 'color'], columns='price_cate', aggfunc='mean', margins=True)

price_cate             저가        중가        고가       All
cut       color                                        
Fair      D      0.380000  0.788667  1.263793  0.920123
          E      0.422045  0.810948  1.238125  0.856607
          F      0.469538  0.813401  1.321800  0.904712
          G      0.520667  0.841524  1.524190  1.023822
          H      0.528571  0.925294  1.680155  1.219175
          I      0.575000  0.917573  1.764516  1.198057
          J      0.512500  0.954848  2.055111  1.341176
Good      D      0.366716  0.732222  1.152030  0.744517
          E      0.368176  0.722384  1.185882  0.745134
          F      0.369234  0.753515  1.190988  0.775930
          G      0.359116  0.778609  1.260220  0.850896
          H      0.357842  0.828496  1.392256  0.914729
          I      0.373898  0.845053  1.622383  1.057222
          J      0.413538  0.910360  1.600229  1.099544
Ideal     D      0.341201  0.594647  1.129029  0.565766
          E      0.341890  0.617754  1.159251  0.578401
          F      0.345013  0.636854  1.206230  0.655829
          G      0.346465  0.623904  1.236651  0.700715
          H      0.349263  0.688589  1.334912  0.799525
          I      0.369817  0.754603  1.482795  0.913029
          J      0.408122  0.843202  1.562033  1.063594
Premium   D      0.352360  0.658421  1.203803  0.721547
          E      0.355189  0.685993  1.241884  0.717745
          F      0.371471  0.725741  1.250113  0.827036
          G      0.359475  0.725989  1.311747  0.841488
          H      0.349070  0.837882  1.417853  1.016449
          I      0.354286  0.876406  1.600363  1.144937
          J      0.385354  0.934612  1.712554  1.293094
Very Good D      0.336331  0.664438  1.137445  0.696424
          E      0.328610  0.676920  1.171712  0.676317
          F      0.332036  0.711580  1.189147  0.740961
          G      0.350347  0.707678  1.233414  0.766799
          H      0.347018  0.766494  1.359085  0.915948
          I      0.374531  0.816453  1.537214  1.046952
          J      0.421729  0.868326  1.605957  1.133215
All              0.352691  0.709712  1.331734  0.797940